[Back to PyCampNextLevel Outline](PyCampNextLevel.ipynb)

# Context Managers and SQL

<a data-flickr-embed="true"  href="https://www.flickr.com/photos/kirbyurner/4536485960/in/album-72157626755979299/" title="stsci 004"><img src="https://live.staticflickr.com/4044/4536485960_24011f181f.jpg" width="500" height="375" alt="stsci 004"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Context manager types are defined by their two characteristic methods, ```__enter__``` and ```__exit__```.  As a Python programmer, you're free to make up applications for this grammar.  Its purpose is to provide a "scope specific" object you will typically want to open and close at the start and end of the scope, however this is not the only pattern one might use.  Allowing the "scope object" to continue beyond the scope is certainly an option.

Lets check out the pattern, which is based on a class.

In [1]:
class CM:
    
    def __enter__(self):
        print("Entering...")
        self.a = [1,2,3]
        return self  # <--- as self
    
    def __exit__(self, *oops):
        """
        If an exception occurs in the scope (indented block)
        then instead of None, None, None coming into __exit__,
        will be about the details of the exception. *oops scoops
        the three arguments into a single tuple, however this is
        not the required parameter pattern.  Just deal with three
        arguments.
        """
        if oops[0]:
            print("Exception in play...")
            print("Handling it...")
            return True
        print("Exiting")

In [2]:
with CM() as obj:
    print("Within the scope {}".format(obj.a))
    
print("obj is still alive: {}".format(obj.a))

Entering...
Within the scope [1, 2, 3]
Exiting
obj is still alive: [1, 2, 3]


In [3]:
with CM() as obj:
    print("Within the scope {}".format(obj.a))
    raise Exception
    
print("obj is still alive: {}".format(obj.a))

Entering...
Within the scope [1, 2, 3]
Exception in play...
Handling it...
obj is still alive: [1, 2, 3]


### Hexworld Game

```hexworld.py``` uses a lot of Python keywords and constructs, including the context manager feature.  The Game class has ```__enter__``` and ```__exit__``` methods to help structure the flow.

In [4]:
import hexworld
help(hexworld.Game)

Help on class Game in module hexworld:

class Game(builtins.object)
 |  Game(player)
 |  
 |  Will the player score more than 100 points before the 
 |  allowed number of turns, max_turns, runs out?
 |  
 |  Designed for use in a try block with a while True loop.
 |  The only way to escape the loop is by means of an 
 |  exception.  However Quitter is handled by __exit__
 |  whereas Winner and Loser propagate outside the context.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |      As you enter a context, you must go through here
 |  
 |  __exit__(self, *oops)
 |      As you leave a context, you must go through here
 |  
 |  __init__(self, player)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  turn_to_play(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak refe

### Airports With SQL

```airports.db``` is a SQLite database, which is basically a text file designed to work with the standard SQL database API, called the DBI.

Lets call out to the operating system just to get some stats on the file.

In [5]:
! ls -g ./data/airports.*

-rwxr-xr-x@ 1 staff  475136 Apr  2 22:15 ./data/airports.db


We have another way of looking into a file's details, through the ```os``` module.

In [6]:
import os
r = os.stat("./data/airports.db")
r.st_size

475136

OK, lets turn to using the ```sqlite3``` module in the Standard Library.

In [7]:
import sqlite3 as sql

In [8]:
type(sql)

module

In [9]:
con = sql.connect("./data/airports.db")

In [10]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Airports',)]


In [11]:
result = cursor.execute("PRAGMA table_info('Airports')").fetchall()
result

[(0, 'iata', 'text', 0, None, 0),
 (1, 'iso', 'text', 0, None, 0),
 (2, 'name', 'text', 0, None, 0),
 (3, 'continent', 'text', 0, None, 0),
 (4, 'type', 'text', 0, None, 0),
 (5, 'lat', 'float', 0, None, 0),
 (6, 'lon', 'float', 0, None, 0),
 (7, 'size', 'text', 0, None, 0),
 (8, 'status', 'int', 0, None, 0)]

In [12]:
result = cursor.execute("SELECT * FROM Airports WHERE iata='SFO'").fetchall()

In [13]:
result

[('SFO',
  'US',
  'San Francisco International Airport',
  'NA',
  'airport',
  37.615215,
  -122.38988,
  'large',
  1)]

In [14]:
result = cursor.execute("SELECT * FROM Airports WHERE iso='US'")

In [15]:
us_airports = result.fetchall()
# print(us_airports)
us_airports[10]

('AYZ', 'US', "Zahn's Airport", 'NA', 'closed', None, None, None, 0)

<a data-flickr-embed="true"  href="https://www.flickr.com/photos/kirbyurner/5723981674/in/album-72157626755979299/" title="Shops and Dining at PDX"><img src="https://live.staticflickr.com/3310/5723981674_159335154b.jpg" width="500" height="375" alt="Shops and Dining at PDX"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

In [16]:
import sqlite3 as sql

class Airport:
    """
    Context Manage designed to retrieve data from airports.db 
    as a tuple, for use in scope.  The database remains open
    until the scope is exited.
    """
    
    def __init__(self, code):
        self.code = code # e.g. SFO, PDX...
    
    def __enter__(self):
        self.connect = sql.connect("./data/airports.db")
        self.cursor  = self.connect.cursor()
        # use a tuple to substitute into ? placeholders
        results = self.cursor.execute(
            "SELECT * FROM Airports WHERE iata= ?", (self.code,))
        self.data = results.fetchall()
        return self
    
    def __exit__(self, *oops):
        # no error handling yet
        self.connect.close()

In [17]:
with Airport("PDX") as airport:
    print(airport.data)
    print("indented part")
    print("the context")
print("context")

[('PDX', 'US', 'Portland International Airport', 'NA', 'airport', 45.588997, -122.5929, 'large', 1)]
indented part
the context
context


In [18]:
with Airport("PDX") as airport:
    print(airport.data)

[('PDX', 'US', 'Portland International Airport', 'NA', 'airport', 45.588997, -122.5929, 'large', 1)]


Note that the ```airport``` object keeps a live connection and cursor throughout the scope of the context.

In [19]:
with Airport("LAX") as airport:
    airport.cursor.execute(
        "SELECT name FROM Airports WHERE iata = ?", 
        ("PDX",))  # or any arbitrary airport, just to show this degree of freedom
    print(airport.cursor.fetchall())

[('Portland International Airport',)]


[Back to PyCampNextLevel Outline](PyCampNextLevel.ipynb)